In [2]:
!pip install --upgrade --quiet pinecone-client pinecone-text pinecone-notebooks


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
api_key = os.getenv("PINECONE_API_KEY")

In [5]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [8]:
from pinecone import Pinecone, ServerlessSpec
index_name = 'hybrid-search-index'
pc = Pinecone(api_key=api_key)

if index_name not in pc.list_indexes().names():
    pc.create_index(
    name = index_name,
    dimension = 384,
    metric='dotproduct', # sparse values supported for dot product
    spec=ServerlessSpec(cloud= 'aws', region='us-east-1')
    )




In [9]:
index = pc.Index(index_name)
index

In [10]:
# vector embeddings and sparse matrix

from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder().default()

[nltk_data] Downloading package punkt to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
sentences = [
    "Exploring the vibrant streets of Tokyo, I discovered a fusion of traditional culture and modern technology.",
"The serene beaches of Bali offer a perfect escape for relaxation and adventure, with stunning sunsets and rich local traditions.",
"Traveling through the historical landmarks of Rome, I was captivated by the ancient architecture and the stories behind each monument."
]

In [14]:
# Applying TfIDF values on these sentences
bm25_encoder.fit(sentences)

# store the values to a json file
bm25_encoder.dump("bm25_values.json")

  0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
# load to your BM25Encoder object

bm25_encoder1 = BM25Encoder().load("bm25_values.json")

In [17]:
retriever = PineconeHybridSearchRetriever(
    embeddings = embeddings,
    sparse_encoder = bm25_encoder,
    index=index
)

In [18]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x7fe3417e3550>, index=<pinecone.data.index.Index object at 0x7fe44da54df0>)

In [19]:
retriever.add_texts(sentences)

  0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
retriever.invoke("What I discoverd in Tokyo?")

[Document(metadata={'score': 0.46285376}, page_content='Exploring the vibrant streets of Tokyo, I discovered a fusion of traditional culture and modern technology.'),
 Document(metadata={'score': 0.151352093}, page_content='Traveling through the historical landmarks of Rome, I was captivated by the ancient architecture and the stories behind each monument.'),
 Document(metadata={'score': 0.0685436502}, page_content='The serene beaches of Bali offer a perfect escape for relaxation and adventure, with stunning sunsets and rich local traditions.')]